# Goal
1. Scrape fashion products data from https://www.fordays.com.
2. Store the data in a hosted Postgres database.

# Scraping Strategy
### About the Website
There are generally **3 product categories**: Women, Men and Baby + Kids. Although there is a tab called "For All", it is a subset of all the products, so we had better scrape the 3 categories respectively for the whole website.

We can treat the website as **3 levels**: home page, product page and detail page. 
- **Home page**: 3 product genders
- **Product page**: overviews of proudcts in one category, like name and price
- **Detail page**: detailed information of one product, including size, color, inventory, description, fabric, etc

### Database Structure
**Rows**: Each row represents **one sku**, which means a product with specific color and size. This allows us to track the invertory information for each sku. 

**Fields**:
```
product_id (str)
display_name (str)
color (str)
size (str)
gender (str) [Men, Women or Baby + Kids]
sale_price (str) [actual sale price after discounts or promotions]
regular_price (str) [original price before discounts or promotions]
product_url (str)
description (str)
is_available (bool) [False if the product is sold out]
fit (str) [a brief description of product fit]
fabric (str) [a brief description of product fabric]
sustainability (str) [a brief description of product sustainability]
recycle (str) [a brief description of product recycle]
scrapped_date (date)
```

# Web Scraping Test

In [1]:
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

import pandas as pd
import numpy as np
import time
from datetime import date
import re

import psycopg2
from sqlalchemy import create_engine

import warnings
warnings.filterwarnings(action='ignore')

pd.set_option('display.max_columns', 100)

### Home Page
Get links of product pages for 3 categories.

In [2]:
#home page
url = 'https://fordays.com/'

def home_page(url):
    try:
        response = requests.get(url)
    except:
        print('Request Failure')

    #get url of 3 categories: Women, Men, Baby + Kids
    try:
        home_soup = BeautifulSoup(response.content, 'lxml') #set up bs object, parser=lxml
        home_nav = home_soup.find_all('a', {'class': 'site-nav__link site-nav__link--underline site-nav__link--has-dropdown'})
        home_url = {home_nav[0].get_text().strip(): 'https://fordays.com' + home_nav[0].get('href'), 
                    home_nav[1].get_text().strip(): 'https://fordays.com' + home_nav[1].get('href'), 
                    home_nav[3].get_text().strip(): 'https://fordays.com' + home_nav[3].get('href')}
        return home_url
    except:
        print('Scraping Error')

In [3]:
home_url = home_page(url)
home_url

{'Women': 'https://fordays.com/collections/all-women',
 'Men': 'https://fordays.com/collections/all-men',
 'Baby + Kids': 'https://fordays.com/collections/all-baby-and-kids'}

### Product Page Example: Men Products Page 1
Product page is the page showing different products. Here we can get **displayed_name, product_url, sale_price and regular_price**. 

Selenium is needed here, since if there is a discount or other promotions, the actual sale price will be different from regular price and will be dynamic, bs4 is not enough. Therefore, here **I first scrape the source page using Selenium, then load it into BeautifulSoup for further analysis**.

In [4]:
#set up Selenium webdriver
#scraping Men products as an example
def product_page(page_url, gender):
    #set up Selenium webdriver
    try:
        driver = webdriver.Chrome(ChromeDriverManager().install())
        driver.get(page_url)
        time.sleep(5)
        page = driver.page_source
        soup = BeautifulSoup(page, 'lxml')
        driver.close()
    except:
        print('Driver Failure')
    else:
        try:
            #find all product_id, display_name, product_url, prices
            product_id_list = soup.find_all('div', {'data-aos': 'row-of-3'})
            display_name_list = soup.find_all('div', {'class': 'grid-product__title text-sm-h4 text-lg-h5'})
            product_url_list = soup.find_all('a', {'class': 'grid-product__link'})
            price_containers = soup.find_all('div', {'class': 'price-container'})

            #extract data records
            products_list = []
            for i in range(len(display_name_list)):
                #gender, product_id, display_name, product_url
                product_gender = gender
                product_id = product_id_list[i].get('data-product-id')
                display_name = display_name_list[i].get_text()
                product_url = 'https://fordays.com' + product_url_list[i].get('href')

                #sale_price, regular_price
                sale_price = None
                regular_price = None
                prices = []
                for e in re.split('\$|\n', price_containers[i].get_text().strip()):
                    try:
                        prices.append(float(e)) #If an element can be converted to float, it is a price. Otherwise, it is other kinds of string.
                    except:
                        pass

                if len(prices) == 1: #Only 1 price element found, sale price = regular price
                    sale_price = '$' + format(prices[0], '.2f')
                    regular_price = '$' + format(prices[0], '.2f')
                else:
                    sale_price = '$' + format(min(prices), '.2f') #the minimun one is sale price
                    regular_price = '$' + format(max(prices), '.2f') #the maximun one is regular price

                products_list.append((product_id, display_name, product_gender, product_url, sale_price, regular_price))

            return products_list
        except:
            print('Scraping Failure')

In [5]:
test_url = home_url['Men'] + '?page=1'
products_list = product_page(test_url, 'Men')

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/102.0.5005.61/chromedriver_mac64_m1.zip
[WDM] - Driver has been saved in cache [/Users/zhaogao/.wdm/drivers/chromedriver/mac64_m1/102.0.5005.61]


In [6]:
pd.DataFrame(products_list, columns=['product_id', 'display_name', 'gender', 'product_url', 'sale_price', 'regular_price'])

,product_id,display_name,gender,product_url,sale_price,regular_price
0,7222375415960,Men's Everlasting Field Jacket,Men,https://fordays.com/collections/all-men/produc...,$138.00,$138.00
1,7222377709720,Men's Everlasting Jogger,Men,https://fordays.com/collections/all-men/produc...,$118.00,$118.00
2,7243347787928,Men's Recycled Fleece Hoodie,Men,https://fordays.com/collections/all-men/produc...,$88.00,$88.00
3,7243343200408,Men's Recycled Fleece Jogger,Men,https://fordays.com/collections/all-men/produc...,$78.00,$78.00
4,7243346608280,Men's Recycled Fleece Sweatshirt,Men,https://fordays.com/collections/all-men/produc...,$78.00,$78.00
5,7243344052376,Men's Recycled Fleece Jogger Short,Men,https://fordays.com/collections/all-men/produc...,$48.00,$48.00
6,7243351621784,Men's Recycled Fleece Oversized Tee,Men,https://fordays.com/collections/all-men/produc...,$52.00,$52.00
7,7243359649944,Men's No Worries Jersey Long Sleeve,Men,https://fordays.com/collections/all-men/produc...,$58.00,$58.00
8,7243361812632,Men's No Worries Jersey Muscle Tank,Men,https://fordays.com/collections/all-men/produc...,$44.00,$44.00
9,7099809136792,Re-New For Better Days Sweatshirt,Men,https://fordays.com/collections/all-men/produc...,$68.00,$68.00


### Detail Page
Detail page includes detailed information of each product, including **color, size, is_available, description, fit, fabric, sustainability and recycle**. In this part, each piece of data represents **one sku**, which means a product with specific color and size. This allows us to store the invertory information for each sku.

Note that for color, size and is_available information, there are some different cases to be considered. A normal case is like "KHAKI / S - \\$138.00 USD". If the product doesn't have a concept of color and size, it will be like "Default Title - \\$138.00 USD". If a product is sold out, it will be like "WHITE / XS - Sold Out".

Selenium is still needed here, since some information is included in buttons, which can't be reached only using bs4.

In [7]:
def detail_page(products_list):
    details_list = []
    for product in products_list:
        #scrapped_date
        scrapped_date = str(date.today())

        #product_id, display_name, product_gender, product_url, sale_price, regular_price
        product_id = product[0]
        display_name = product[1]
        product_gender = product[2]
        product_url = product[3]
        sale_price = product[4]
        regular_price = product[5]

        #set up Selenium webdriver
        try:
            driver = webdriver.Chrome(ChromeDriverManager().install())
            driver.get(product_url)
            page = driver.page_source
            soup = BeautifulSoup(page, 'lxml')
            driver.close()
        except:
            print('Driver Failure')
        else:
            try:
                #description
                description_rte = soup.find('div', {'class': 'product-single__description rte'})
                if len(description_rte.find_all('h2')) == 0: #e.g. https://fordays.com/collections/all-men/products/for-take-out-tote
                    description = description_rte.get_text().strip()
                else:
                    description = description_rte.find('h2').get_text().strip()

                #fit, fabric, sustainability, recycle
                fit = None
                fabric = None
                sustainability = None
                recycle = None

                buttons = soup.find_all('button', {'class': 'product-collapsible'}) #titles
                contents = soup.find_all('div', {'class': 'product-collapsible-content'}) #contents
                if len(buttons) > 0:
                    for i in range(len(buttons)):
                        title = buttons[i].find('h6').get_text()
                        if title == 'Fabric': 
                            fabric = contents[i].find('p').get_text()
                        elif title == 'Fit': 
                            fit = contents[i].find('p').get_text()
                        elif title == 'Sustainability':
                            sustainability = contents[i].find('p').get_text()
                        elif title == 'Recycle & Earn':
                            recycle = contents[i].find('p').get_text()

                #color, size, inventory
                ##normal case: KHAKI / S - $138.00 USD
                ##special cases: Default Title - Sold Out, WHITE / XS - Sold Out
                skus = soup.find('select', {'class': 'product-single__variants no-js'})
                options = skus.find_all('option')

                for i in range(len(options)):
                    sku_content = re.split(' / | - ', options[i].get_text().strip())
                    color = None
                    size = None
                    is_available = True
                    if sku_content[0] != 'Default Title':
                        color = sku_content[0]
                        size = sku_content[1]
                    if sku_content[-1] == 'Sold Out':
                        is_available = False

                    details_list.append((product_id, display_name, color, size, product_gender, sale_price, regular_price, 
                                         product_url, description, is_available, fit, fabric, sustainability, recycle, 
                                         scrapped_date))        
            except:
                print('Scraping Failure')
        
    return details_list

In [8]:
details_list = detail_page(products_list)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [/Users/zhaogao/.wdm/drivers/chromedriver/mac64_m1/102.0.5005.61/chromedriver] found in cache


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [/Users/zhaogao/.wdm/drivers/chromedriver/mac64_m1/102.0.5005.61/chromedriver] found in cache


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [/Users/zhaogao/.wdm/drivers/chromedriver/mac64_m1/102.0.5005.61/chromedriver] found in cache


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [/Users/zhaogao/.wdm/drivers/chromedriver/mac64_m1/102.0.5005.61/chromedriver] found in cache


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [/Users/zhaogao/.wdm/drivers/chromedriver/mac64_m1/102.0.5005.61/chromedriver] found in cache


...

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [/Users/zhaogao/.wdm/drivers/chromedriver/mac64_m1/102.0.5005.61/chromedriver] found in cache


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [/Users/zhaogao/.wdm/drivers/chromedriver/mac64_m1/102.0.5005.61/chromedriver] found in cache


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [/Users/zhaogao/.wdm/drivers/chromedriver/mac64_m1/102.0.5005.61/chromedriver] found in cache


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [/Users/zhaogao/.wdm/drivers/chromedriver/mac64_m1/102.0.5005.61/chromedriver] found in cache


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [/Users/zhaogao/.wdm/drivers/chromedriver/mac64_m1/102.0.5005.61/chromedriver] found in cache


In [9]:
df_test = pd.DataFrame(details_list, columns=['product_id', 'display_name', 'color', 'size', 'gender', 'sale_price', 'regular_price', 
                                              'product_url', 'description', 'is_available', 'fit', 'fabric', 'sustainability', 'recycle', 
                                              'scrapped_date'])
df_test

,product_id,display_name,color,size,gender,sale_price,regular_price,product_url,description,is_available,fit,fabric,sustainability,recycle,scrapped_date
0,7222375415960,Men's Everlasting Field Jacket,KHAKI,XS,Men,$138.00,$138.00,https://fordays.com/collections/all-men/produc...,The Everlasting Field Jacket is the vintage-in...,True,Women size down for a true to size fit. Chris...,Organic Cotton Twill is made with 97% organic ...,Zero waste. 100% recyclable fashion. Organic a...,Earn Closet Cash for shopping sustainably and ...,2022-06-03
1,7222375415960,Men's Everlasting Field Jacket,KHAKI,S,Men,$138.00,$138.00,https://fordays.com/collections/all-men/produc...,The Everlasting Field Jacket is the vintage-in...,True,Women size down for a true to size fit. Chris...,Organic Cotton Twill is made with 97% organic ...,Zero waste. 100% recyclable fashion. Organic a...,Earn Closet Cash for shopping sustainably and ...,2022-06-03
2,7222375415960,Men's Everlasting Field Jacket,KHAKI,M,Men,$138.00,$138.00,https://fordays.com/collections/all-men/produc...,The Everlasting Field Jacket is the vintage-in...,True,Women size down for a true to size fit. Chris...,Organic Cotton Twill is made with 97% organic ...,Zero waste. 100% recyclable fashion. Organic a...,Earn Closet Cash for shopping sustainably and ...,2022-06-03
3,7222375415960,Men's Everlasting Field Jacket,KHAKI,L,Men,$138.00,$138.00,https://fordays.com/collections/all-men/produc...,The Everlasting Field Jacket is the vintage-in...,True,Women size down for a true to size fit. Chris...,Organic Cotton Twill is made with 97% organic ...,Zero waste. 100% recyclable fashion. Organic a...,Earn Closet Cash for shopping sustainably and ...,2022-06-03
4,7222375415960,Men's Everlasting Field Jacket,KHAKI,XL,Men,$138.00,$138.00,https://fordays.com/collections/all-men/produc...,The Everlasting Field Jacket is the vintage-in...,True,Women size down for a true to size fit. Chris...,Organic Cotton Twill is made with 97% organic ...,Zero waste. 100% recyclable fashion. Organic a...,Earn Closet Cash for shopping sustainably and ...,2022-06-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458,7151885353112,Poplin Pant,GARNET,S,Men,$54.00,$54.00,https://fordays.com/collections/all-men/produc...,Feel like you actually got dressed for WFH. Ca...,True,Relaxed throughout the leg. Pull-on elastic wa...,Lightweight 100% organic cotton poplin. Think ...,Zero waste. 100% recyclable fashion. Organic a...,Earn Closet Cash for shopping sustainably and ...,2022-06-03
459,7151885353112,Poplin Pant,GARNET,M,Men,$54.00,$54.00,https://fordays.com/collections/all-men/produc...,Feel like you actually got dressed for WFH. Ca...,True,Relaxed throughout the leg. Pull-on elastic wa...,Lightweight 100% organic cotton poplin. Think ...,Zero waste. 100% recyclable fashion. Organic a...,Earn Closet Cash for shopping sustainably and ...,2022-06-03
460,7151885353112,Poplin Pant,GARNET,L,Men,$54.00,$54.00,https://fordays.com/collections/all-men/produc...,Feel like you actually got dressed for WFH. Ca...,True,Relaxed throughout the leg. Pull-on elastic wa...,Lightweight 100% organic cotton poplin. Think ...,Zero waste. 100% recyclable fashion. Organic a...,Earn Closet Cash for shopping sustainably and ...,2022-06-03
461,7151885353112,Poplin Pant,GARNET,XL,Men,$54.00,$54.00,https://fordays.com/collections/all-men/produc...,Feel like you actually got dressed for WFH. Ca...,True,Relaxed throughout the leg. Pull-on elastic wa...,Lightweight 100% organic cotton poplin. Think ...,Zero waste. 100% recyclable fashion. Organic a...,Earn Closet Cash for shopping sustainably and ...,2022-06-03


### Check Data Quality
- There is one special case where the locations of size and color are reversed. 
- We can fix it here, but this part can't be automated yet.

In [10]:
df_test[df_test['size']=='BLACK']

,product_id,display_name,color,size,gender,sale_price,regular_price,product_url,description,is_available,fit,fabric,sustainability,recycle,scrapped_date
404,6286270333080,Take Back Bag,L,BLACK,Men,$20.00,$20.00,https://fordays.com/collections/all-men/produc...,The Take Back Bag is here to start a circular ...,True,Large Bag measures 24 in x 24 in and fits 25 lbs,None,Each TBB Saves: 197 lbs of CO2 emissions 27K...,None,2022-06-03


In [11]:
df_test.loc[404, 'size'] = 'L'
df_test.loc[404, 'color'] = 'BLACK'

In [12]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   product_id      463 non-null    object
 1   display_name    463 non-null    object
 2   color           462 non-null    object
 3   size            462 non-null    object
 4   gender          463 non-null    object
 5   sale_price      463 non-null    object
 6   regular_price   463 non-null    object
 7   product_url     463 non-null    object
 8   description     463 non-null    object
 9   is_available    463 non-null    bool  
 10  fit             462 non-null    object
 11  fabric          461 non-null    object
 12  sustainability  462 non-null    object
 13  recycle         461 non-null    object
 14  scrapped_date   463 non-null    object
dtypes: bool(1), object(14)
memory usage: 51.2+ KB


# Store Data in Database

### Local Database
We can see that data has been stored in the local database using psql.

In [13]:
#store data to database
def store_to_db(df, table_name, host, port, user, passw, database):
    try:
        #connect to database
        conn_string = 'postgresql://' + user + ':' + passw + '@' + host + ':' + str(port) + '/' + database
        db = create_engine(conn_string, echo=False)
        conn = db.connect()

        #store data
        df.to_sql('products', con=conn, if_exists='replace', index=False)
        conn.close()
        print('Storage Success')
    except:
        print('Storage Error')

#query data from database
def query_from_db(table_name, host, port, user, passw, database):
    db_data = []
    try:
        conn_string = 'postgresql://' + user + ':' + passw + '@' + host + ':' + str(port) + '/' + database
        db = create_engine(conn_string, echo=False)
        conn = psycopg2.connect(conn_string)
        conn.autocommit = True
        cursor = conn.cursor()

        sql_str = '''select * from products;'''
        cursor.execute(sql_str)
        
        for i in cursor.fetchall():
            db_data.append(i)

        # conn.commit()
        conn.close()
        print('Query Success')
    except:
        print('Query Error')
    return db_data

In [14]:
host = 'localhost'
port = int(5432)
user = 'postgres'
passw = 'myPassword'
database = 'fashion'

store_to_db(df_test, 'product', host, port, user, passw, database)

Storage Success


In [15]:
data_local = query_from_db('products', host, port, user, passw, database)
pd.DataFrame(data_local, columns=['product_id', 'display_name', 'color', 'size', 'gender', 'sale_price', 'regular_price', 
                                  'product_url', 'description', 'is_available', 'fit', 'fabric', 'sustainability', 'recycle', 
                                  'scrapped_date'])

Query Success


,product_id,display_name,color,size,gender,sale_price,regular_price,product_url,description,is_available,fit,fabric,sustainability,recycle,scrapped_date
0,7222375415960,Men's Everlasting Field Jacket,KHAKI,XS,Men,$138.00,$138.00,https://fordays.com/collections/all-men/produc...,The Everlasting Field Jacket is the vintage-in...,True,Women size down for a true to size fit. Chris...,Organic Cotton Twill is made with 97% organic ...,Zero waste. 100% recyclable fashion. Organic a...,Earn Closet Cash for shopping sustainably and ...,2022-06-03
1,7222375415960,Men's Everlasting Field Jacket,KHAKI,S,Men,$138.00,$138.00,https://fordays.com/collections/all-men/produc...,The Everlasting Field Jacket is the vintage-in...,True,Women size down for a true to size fit. Chris...,Organic Cotton Twill is made with 97% organic ...,Zero waste. 100% recyclable fashion. Organic a...,Earn Closet Cash for shopping sustainably and ...,2022-06-03
2,7222375415960,Men's Everlasting Field Jacket,KHAKI,M,Men,$138.00,$138.00,https://fordays.com/collections/all-men/produc...,The Everlasting Field Jacket is the vintage-in...,True,Women size down for a true to size fit. Chris...,Organic Cotton Twill is made with 97% organic ...,Zero waste. 100% recyclable fashion. Organic a...,Earn Closet Cash for shopping sustainably and ...,2022-06-03
3,7222375415960,Men's Everlasting Field Jacket,KHAKI,L,Men,$138.00,$138.00,https://fordays.com/collections/all-men/produc...,The Everlasting Field Jacket is the vintage-in...,True,Women size down for a true to size fit. Chris...,Organic Cotton Twill is made with 97% organic ...,Zero waste. 100% recyclable fashion. Organic a...,Earn Closet Cash for shopping sustainably and ...,2022-06-03
4,7222375415960,Men's Everlasting Field Jacket,KHAKI,XL,Men,$138.00,$138.00,https://fordays.com/collections/all-men/produc...,The Everlasting Field Jacket is the vintage-in...,True,Women size down for a true to size fit. Chris...,Organic Cotton Twill is made with 97% organic ...,Zero waste. 100% recyclable fashion. Organic a...,Earn Closet Cash for shopping sustainably and ...,2022-06-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458,7151885353112,Poplin Pant,GARNET,S,Men,$54.00,$54.00,https://fordays.com/collections/all-men/produc...,Feel like you actually got dressed for WFH. Ca...,True,Relaxed throughout the leg. Pull-on elastic wa...,Lightweight 100% organic cotton poplin. Think ...,Zero waste. 100% recyclable fashion. Organic a...,Earn Closet Cash for shopping sustainably and ...,2022-06-03
459,7151885353112,Poplin Pant,GARNET,M,Men,$54.00,$54.00,https://fordays.com/collections/all-men/produc...,Feel like you actually got dressed for WFH. Ca...,True,Relaxed throughout the leg. Pull-on elastic wa...,Lightweight 100% organic cotton poplin. Think ...,Zero waste. 100% recyclable fashion. Organic a...,Earn Closet Cash for shopping sustainably and ...,2022-06-03
460,7151885353112,Poplin Pant,GARNET,L,Men,$54.00,$54.00,https://fordays.com/collections/all-men/produc...,Feel like you actually got dressed for WFH. Ca...,True,Relaxed throughout the leg. Pull-on elastic wa...,Lightweight 100% organic cotton poplin. Think ...,Zero waste. 100% recyclable fashion. Organic a...,Earn Closet Cash for shopping sustainably and ...,2022-06-03
461,7151885353112,Poplin Pant,GARNET,XL,Men,$54.00,$54.00,https://fordays.com/collections/all-men/produc...,Feel like you actually got dressed for WFH. Ca...,True,Relaxed throughout the leg. Pull-on elastic wa...,Lightweight 100% organic cotton poplin. Think ...,Zero waste. 100% recyclable fashion. Organic a...,Earn Closet Cash for shopping sustainably and ...,2022-06-03


### AWS RDS
We can see in pyAdmin that data has stored in AWS RDS.

In [16]:
#store data to database
host = 'fashion.ctuxfwqupsvc.us-east-1.rds.amazonaws.com'
port = int(5432)
user = 'postgres'
passw = 'myPassword'
database = 'postgres'

store_to_db(df_test, 'product', host, port, user, passw, database)

Storage Success


In [17]:
data_aws = query_from_db('products', host, port, user, passw, database)
pd.DataFrame(data_aws, columns=['product_id', 'display_name', 'color', 'size', 'gender', 'sale_price', 'regular_price', 
                                'product_url', 'description', 'is_available', 'fit', 'fabric', 'sustainability', 'recycle', 
                                'scrapped_date'])

Query Success


,product_id,display_name,color,size,gender,sale_price,regular_price,product_url,description,is_available,fit,fabric,sustainability,recycle,scrapped_date
0,7222375415960,Men's Everlasting Field Jacket,KHAKI,XS,Men,$138.00,$138.00,https://fordays.com/collections/all-men/produc...,The Everlasting Field Jacket is the vintage-in...,True,Women size down for a true to size fit. Chris...,Organic Cotton Twill is made with 97% organic ...,Zero waste. 100% recyclable fashion. Organic a...,Earn Closet Cash for shopping sustainably and ...,2022-06-03
1,7222375415960,Men's Everlasting Field Jacket,KHAKI,S,Men,$138.00,$138.00,https://fordays.com/collections/all-men/produc...,The Everlasting Field Jacket is the vintage-in...,True,Women size down for a true to size fit. Chris...,Organic Cotton Twill is made with 97% organic ...,Zero waste. 100% recyclable fashion. Organic a...,Earn Closet Cash for shopping sustainably and ...,2022-06-03
2,7222375415960,Men's Everlasting Field Jacket,KHAKI,M,Men,$138.00,$138.00,https://fordays.com/collections/all-men/produc...,The Everlasting Field Jacket is the vintage-in...,True,Women size down for a true to size fit. Chris...,Organic Cotton Twill is made with 97% organic ...,Zero waste. 100% recyclable fashion. Organic a...,Earn Closet Cash for shopping sustainably and ...,2022-06-03
3,7222375415960,Men's Everlasting Field Jacket,KHAKI,L,Men,$138.00,$138.00,https://fordays.com/collections/all-men/produc...,The Everlasting Field Jacket is the vintage-in...,True,Women size down for a true to size fit. Chris...,Organic Cotton Twill is made with 97% organic ...,Zero waste. 100% recyclable fashion. Organic a...,Earn Closet Cash for shopping sustainably and ...,2022-06-03
4,7222375415960,Men's Everlasting Field Jacket,KHAKI,XL,Men,$138.00,$138.00,https://fordays.com/collections/all-men/produc...,The Everlasting Field Jacket is the vintage-in...,True,Women size down for a true to size fit. Chris...,Organic Cotton Twill is made with 97% organic ...,Zero waste. 100% recyclable fashion. Organic a...,Earn Closet Cash for shopping sustainably and ...,2022-06-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458,7151885353112,Poplin Pant,GARNET,S,Men,$54.00,$54.00,https://fordays.com/collections/all-men/produc...,Feel like you actually got dressed for WFH. Ca...,True,Relaxed throughout the leg. Pull-on elastic wa...,Lightweight 100% organic cotton poplin. Think ...,Zero waste. 100% recyclable fashion. Organic a...,Earn Closet Cash for shopping sustainably and ...,2022-06-03
459,7151885353112,Poplin Pant,GARNET,M,Men,$54.00,$54.00,https://fordays.com/collections/all-men/produc...,Feel like you actually got dressed for WFH. Ca...,True,Relaxed throughout the leg. Pull-on elastic wa...,Lightweight 100% organic cotton poplin. Think ...,Zero waste. 100% recyclable fashion. Organic a...,Earn Closet Cash for shopping sustainably and ...,2022-06-03
460,7151885353112,Poplin Pant,GARNET,L,Men,$54.00,$54.00,https://fordays.com/collections/all-men/produc...,Feel like you actually got dressed for WFH. Ca...,True,Relaxed throughout the leg. Pull-on elastic wa...,Lightweight 100% organic cotton poplin. Think ...,Zero waste. 100% recyclable fashion. Organic a...,Earn Closet Cash for shopping sustainably and ...,2022-06-03
461,7151885353112,Poplin Pant,GARNET,XL,Men,$54.00,$54.00,https://fordays.com/collections/all-men/produc...,Feel like you actually got dressed for WFH. Ca...,True,Relaxed throughout the leg. Pull-on elastic wa...,Lightweight 100% organic cotton poplin. Think ...,Zero waste. 100% recyclable fashion. Organic a...,Earn Closet Cash for shopping sustainably and ...,2022-06-03
